In [1]:
import gymnasium as gym
from urdfenvs.urdf_common.urdf_env import UrdfEnv
from urdfenvs.robots.generic_urdf import GenericUrdfReacher
from urdfenvs.sensors.full_sensor import FullSensor
from urdfenvs.scene_examples.goal import goal1

import numpy as np

import os

pybullet build time: May 20 2022 19:45:31


In [2]:
from forwardkinematics.urdfFks.pointRobotUrdfFk import PointRobotUrdfFk
from urdfenvs.urdf_common.reward import Reward


class SampleReward(Reward):
    def calculate_reward(self, observation: dict) -> float:
        goalPosition = observation['robot_0']['FullSensor']['goals'][1]['position']
        robotPosition = observation['robot_0']['joint_state']['position']

        distance = np.linalg.norm(robotPosition - goalPosition)
        # reward = (1 / distance)
        reward = - distance
        print(f"🎁 Reward: {reward}")
        return reward
    

In [3]:
robots = [
    GenericUrdfReacher(urdf="pointRobot.urdf", mode="tor"),
]
render: bool = True

env = UrdfEnv(render=render, robots=robots)

env.add_goal(goal1)

sensor = FullSensor(['position'], ['position', 'size'], variance=0.0)
env.add_sensor(sensor, [0])
env.set_reward_calculator(SampleReward())
env.set_spaces()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (NVIDIA GeForce RTX 3050 Ti Laptop GPU)
GL_VERSION=3.3 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=3.30
pthread_getconcurrency()=0
Version = 3.3 (Core Profile) Mesa 21.2.6
Vendor = Microsoft Corporation
Renderer = D3D12 (NVIDIA GeForce RTX 3050 Ti Laptop GPU)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Microsoft Corporation
ven = Microsoft Corporation


GL error 0x500 detected in glGenBuffers
/home/behradx/anaconda3/envs/SB3/lib/python3.9/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float64
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
env.reset()

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: lidar_sensor_link


({'robot_0': {'joint_state': {'position': array([0., 0., 0.]),
    'velocity': array([0., 0., 0.])},
   'FullSensor': {'goals': {1: {'position': array([1. , 0. , 0.1], dtype=float32)}}}}},
 {})

In [5]:
# while True:
#     action = np.zeros_like(env.action_space.sample())
#     obs, reward, done, truncated, info = env.step(action)

In [7]:
defaultAction = np.array([ 0.16851664, -0.38915175, -0.09081227])
pos0 = np.array([0.0, 0.0, 0.0])
vel0 = np.array([0.0, 0.0, 0.0])

# initial_observations = []
for _ in range(100):
    ob = env.reset(pos=pos0, vel=vel0)
    # env.shuffle_goals()
    # env.shuffle_obstacles()
    # initial_observations.append(ob)
    print(f"Initial observation : {ob}")
    #assert np.array_equal(initial_observations[0], ob)

    # history = []
    for _ in range(250):
        # env.render()
        action = defaultAction
        ob, *_  = env.step(action)
        # In observations, information about obstacles is stored in ob['obstacleSensor']
        # history.append(ob)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: lidar_sensor_link
Initial observation : ({'robot_0': {'joint_state': {'position': array([0., 0., 0.]), 'velocity': array([0., 0., 0.])}, 'FullSensor': {'goals': {1: {'position': array([1. , 0. , 0.1], dtype=float32)}}}}}, {})
🎁 Reward: -1.0049875631336
🎁 Reward: -1.00498756400684
🎁 Reward: -1.00498756488008
🎁 Reward: -1.0049875657533198
🎁 Reward: -1.00498756662656
🎁 Reward: -1.0049875674998001
🎁 Reward: -1.0049875683730403
🎁 Reward: -1.0049875692462806
🎁 Reward: -1.004987570119521
🎁 Reward: -1.0049875709927614
🎁 Reward: -1.004987571866002
🎁 Reward: -1.0049875727392426
🎁 Reward: -1.0049875736124831
🎁 Reward: -1.004987574485724
🎁 Reward: -1.0049875753589648
🎁 Reward: -1.0049875762322056
🎁 Reward: -1.0049

KeyboardInterrupt: 

In [ ]:
# from normalize_action import 
from gymnasium import spaces
import numpy as np


class MapActionWrapper(gym.ActionWrapper):
    def __init__(self, env, low=-1.0, high=1.0):
        super().__init__(env)
        self.low = low 
        self.high = high
        self.action_space = spaces.Box(low=low, high=high, shape=env.action_space.shape, dtype=np.float32)

    def action(self, action):
        orig_low = self.env.action_space.low
        orig_high = self.env.action_space.high
        mapped_action = self.low + (action - orig_low) * (self.high - self.low) / (orig_high - orig_low)
        return mapped_action

In [ ]:
from urdfenvs.wrappers.sb3_float32_action_wrapper import SB3Float32ActionWrapper
from gymnasium.wrappers import TimeLimit
from gymnasium.wrappers import FlattenObservation


env = FlattenObservation(env)
env = SB3Float32ActionWrapper(env)
env = MapActionWrapper(env)
env = TimeLimit(env=env, max_episode_steps= 500)

In [ ]:
env.action_space

Box(-1.0, 1.0, (3,), float32)

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="./checkpoint/",
  name_prefix="point_004",
  save_replay_buffer=False,
)

In [ ]:
from stable_baselines3 import TD3


In [ ]:
MODEL_NAME = 'TD3-006'
MODEL_CLASS = TD3

models_dir = 'models/' + MODEL_NAME
logdir = 'tb_logs'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [ ]:


model = TD3("MlpPolicy", 
                    env, 
                    verbose=0, 
                    tensorboard_log= logdir, 
                    learning_rate= 0.001,
                    batch_size= 256, # It's better to be high. Default is 256. 
                    buffer_size= 2_000_000,
                    # gamma= 0.99, # Discount Factor 
                    # action_noise=action_noise, 
                    device='cpu',
                )

In [ ]:
# TIMESTEPS = 15_000

# model.learn(
#     total_timesteps=TIMESTEPS, 
#     log_interval=10, 
#     tb_log_name=MODEL_NAME,
#     progress_bar=True, 
#     callback= checkpoint_callback
# )


In [ ]:
testModel = model.load("./checkpoint/point_004_15000_steps.zip")

In [ ]:


obs = env.reset()[0]
action , _ = testModel.predict(obs)
print(action)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: lidar_sensor_link
[ 0.16851664 -0.38915175 -0.09081227]


In [ ]:
obs = env.reset()[0]

for _ in range(500):
    env.render()
    action , _ = testModel.predict(obs)
    obs, *_ = env.step(action)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: lidar_sensor_link
🎁 Reward: -1.0049875622605302
🎁 Reward: -1.0049875622606994
🎁 Reward: -1.0049875622608686
🎁 Reward: -1.0049875622610378
🎁 Reward: -1.0049875622612068
🎁 Reward: -1.004987562261376
🎁 Reward: -1.0049875622615452
🎁 Reward: -1.0049875622617144
🎁 Reward: -1.0049875622618836
🎁 Reward: -1.0049875622620525
🎁 Reward: -1.0049875622622217
🎁 Reward: -1.004987562262391
🎁 Reward: -1.00498756226256
🎁 Reward: -1.0049875622627291
🎁 Reward: -1.0049875622628983
🎁 Reward: -1.0049875622630675
🎁 Reward: -1.0049875622632367
🎁 Reward: -1.0049875622634057
🎁 Reward: -1.0049875622635749
🎁 Reward: -1.004987562263744
🎁 Reward: -1.0049875622639133
🎁 Reward: -1.0049875622640825
🎁 Reward: -1.0049875622642515
🎁 Reward